In [47]:
import numpy as np
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow import __version__ as tf_version, float32 as tf_float32, Variable
from tensorflow.keras import Sequential, Model
from tensorflow.keras.backend import variable, dot as k_dot, sigmoid, relu
from tensorflow.keras.layers import Dense, Input, Concatenate, Layer
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.utils import plot_model
from tensorflow.random import set_seed as tf_set_seed
from numpy import __version__ as np_version, unique, array, mean, argmax
from numpy.random import seed as np_seed, choice
from pandas import __version__ as pd_version, read_csv, DataFrame, concat
from sklearn import __version__ as sk_version
from sklearn.preprocessing import normalize
print("tensorflow version:", tf_version)
print("numpy version:", np_version)
print("pandas version:", pd_version)
print("scikit-learn version:", sk_version)

tensorflow version: 2.9.1
numpy version: 1.21.5
pandas version: 1.4.2
scikit-learn version: 1.0.2


In [32]:
X_train,Y_train = np.load("X_train.npy"),np.load("Y_train.npy")
X_test,Y_test = np.load("X_test.npy"),np.load("Y_test.npy")

In [38]:
X_train.shape,X_test.shape

((30274, 2, 5, 49), (7570, 2, 5, 49))

In [41]:
#[blue_top,red_top,blue_jgl,red_jgl,blue_mid,red_mid,blue_adc,red_adc,blue_sup,red_sup]
def cut_down_data(X):
    blue_top,red_top,blue_jgl,red_jgl,blue_mid,red_mid,blue_adc,red_adc,blue_sup,red_sup = [],[],[],[],[],[],[],[],[],[]
    for x in X:
        blue_top.append(x[0][0])
        red_top.append(x[1][0])
        blue_jgl.append(x[0][1])
        red_jgl.append(x[1][1])
        blue_mid.append(x[0][2])
        red_mid.append(x[1][2])
        blue_adc.append(x[0][3])
        red_adc.append(x[1][3])
        blue_sup.append(x[0][4])
        red_sup.append(x[1][4])
    return(np.array(blue_top),np.array(red_top),np.array(blue_jgl),np.array(red_jgl),np.array(blue_mid),np.array(red_mid),np.array(blue_adc),np.array(red_adc),np.array(blue_sup),np.array(red_sup))

In [42]:
Train_blue_top,Train_red_top,Train_blue_jgl,Train_red_jgl,Train_blue_mid,Train_red_mid,Train_blue_adc,Train_red_adc,Train_blue_sup,Train_red_sup = cut_down_data(X_train)
Test_blue_top,Test_red_top,Test_blue_jgl,Test_red_jgl,Test_blue_mid,Test_red_mid,Test_blue_adc,Test_red_adc,Test_blue_sup,Test_red_sup = cut_down_data(X_test)

In [44]:
Train_data = [Train_blue_top,Train_red_top,Train_blue_jgl,Train_red_jgl,Train_blue_mid,Train_red_mid,Train_blue_adc,Train_red_adc,Train_blue_sup,Train_red_sup]

In [43]:
Test_red_top.shape

(7570, 49)

In [8]:
_matchup_shape = 2 * X_train.shape[-1]
_champion_shape = X_train.shape[-1]

In [25]:
tf_set_seed(1102)
np_seed(1102)

blue_top_input = Input(shape = _champion_shape, name = "Blue_top_input")
red_top_input = Input(shape = _champion_shape, name = "Red_top_input")
top_input = Concatenate(name = "Merged_top_input")([blue_top_input,red_top_input])
top_matchup = Dense(32, activation = 'relu', name="Top_matchup")(top_input)

blue_jgl_input = Input(shape = _champion_shape, name = "Blue_jgl_input")
red_jgl_input = Input(shape = _champion_shape, name = "Red_jgl_input")
jgl_input = Concatenate(name = "Merged_jgl_input")([blue_jgl_input,red_jgl_input])
jgl_matchup = Dense(32, activation = 'relu',name = 'Jgl_matchup')(jgl_input) 

blue_jgl_top_input = Concatenate(name = 'Blue_jgl_top_input')([blue_top_input,blue_jgl_input])
blue_jgl_top = Dense(16,activation='relu', name = 'Blue_jgl_top')(blue_jgl_top_input)
red_jgl_top_input = Concatenate(name = 'Red_jgl_top_input')([red_top_input,red_jgl_input])
red_jgl_top = Dense(16,activation='relu', name = 'Red_jgl_top')(red_jgl_top_input)

jgl_top_input = Concatenate(name = "jgl_top_matchup_input")([blue_jgl_top,red_jgl_top])
jgl_top_matchup = Dense(8, name = 'jgl_top_matchup')(jgl_top_input) 


blue_mid_input = Input(shape = _champion_shape, name = "Blue_mid_input")
red_mid_input = Input(shape = _champion_shape,name = 'Red_mid_input')
mid_input = Concatenate(name='Merged_mid_input')([blue_mid_input,red_mid_input])
mid_matchup = Dense(32, activation = 'relu',name = "Mid_matchup")(mid_input)

blue_jgl_mid_input = Concatenate(name = 'Blue_jgl_mid_input')([blue_mid_input,blue_jgl_input])
blue_jgl_mid = Dense(16,activation='relu',name='Blue_jgl_mid')(blue_jgl_mid_input)
red_jgl_mid_input = Concatenate(name='Red_jgl_mid_input')([red_mid_input,red_jgl_input])
red_jgl_mid = Dense(16,activation='relu',name='Red_jgl_mid')(red_jgl_mid_input)

jgl_mid_input = Concatenate(name='Jgl_mid_matchup_input')([blue_jgl_mid,red_jgl_mid])
jgl_mid_matchup = Dense(8,name='Jgl_mid_matchup')(jgl_mid_input) 



blue_adc_input = Input(shape = _champion_shape,name='Blue_adc_input')
red_adc_input = Input(shape = _champion_shape,name='Red_adc_input')
adc_input = Concatenate(name='Merged_adc_input')([blue_adc_input,red_adc_input])
adc_matchup = Dense(32, activation = 'relu',name='ADC_matchup')(adc_input)

blue_sup_input = Input(shape = _champion_shape,name='Blue_sup_input')
red_sup_input = Input(shape = _champion_shape,name='Red_sup_input')
sup_input = Concatenate(name='Sup_matchup_input')([blue_sup_input,red_sup_input])
sup_matchup = Dense(32, activation = 'relu',name='Sup_matchup')(sup_input)

botlane_input = Concatenate(name='Botlane_matchup_input')([adc_matchup,sup_matchup])
botlane_matchup = Dense(16,activation='relu',name='Botlane_matchup')(botlane_input)

blue_jgl_sup_input = Concatenate(name='Blue_jgl_sup_input')([blue_sup_input,blue_jgl_input])
blue_jgl_sup = Dense(16,activation='relu',name='Blue_jgl_sup')(blue_jgl_sup_input)
red_jgl_sup_input = Concatenate(name='Red_jgl_sup_input')([red_sup_input,red_jgl_input])
red_jgl_sup = Dense(16,activation='relu',name='Red_jgl_sup')(red_jgl_sup_input)

jgl_sup_input = Concatenate(name='Merged_jgl_sup_input')([blue_jgl_sup,red_jgl_sup])
jgl_sup_matchup = Dense(8,name = 'Merged_jgl_sup')(jgl_sup_input) 


all_input = Concatenate(name = "Merged_all_input")([top_matchup,jgl_top_matchup,mid_matchup,jgl_mid_matchup,botlane_matchup,jgl_sup_matchup])
dense_layer = Dense(32,activation='tanh',name='Merged_all')(all_input)

final_layer = Dense(1,activation='sigmoid',name='Output')(dense_layer)


BDC = Model(inputs=[blue_top_input,red_top_input,blue_jgl_input,red_jgl_input,blue_mid_input,red_mid_input,blue_adc_input,red_adc_input,blue_sup_input,red_sup_input],outputs = final_layer, name="BDC")
BDC.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
BDC.summary()

Model: "BDC"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Blue_top_input (InputLayer)    [(None, 49)]         0           []                               
                                                                                                  
 Red_top_input (InputLayer)     [(None, 49)]         0           []                               
                                                                                                  
 Blue_jgl_input (InputLayer)    [(None, 49)]         0           []                               
                                                                                                  
 Red_jgl_input (InputLayer)     [(None, 49)]         0           []                               
                                                                                                

 jgl_top_matchup (Dense)        (None, 8)            264         ['jgl_top_matchup_input[0][0]']  
                                                                                                  
 Mid_matchup (Dense)            (None, 32)           3168        ['Merged_mid_input[0][0]']       
                                                                                                  
 Jgl_mid_matchup (Dense)        (None, 8)            264         ['Jgl_mid_matchup_input[0][0]']  
                                                                                                  
 Botlane_matchup (Dense)        (None, 16)           1040        ['Botlane_matchup_input[0][0]']  
                                                                                                  
 Merged_jgl_sup (Dense)         (None, 8)            264         ['Merged_jgl_sup_input[0][0]']   
                                                                                                  
 Merged_al

In [50]:
BDC.fit(Train_data,Y_train, batch_size=64,epochs=10,validation_split=0.2)

Epoch 1/10
379/379 [==============================] - 22s 59ms/step - loss: 0.6935 - accuracy: 0.5000 - val_loss: 0.6928 - val_accuracy: 0.4973
Epoch 2/10
379/379 [==============================] - 21s 56ms/step - loss: 0.6931 - accuracy: 0.5066 - val_loss: 0.6935 - val_accuracy: 0.4918
Epoch 3/10
379/379 [==============================] - 21s 56ms/step - loss: 0.6930 - accuracy: 0.5090 - val_loss: 0.6929 - val_accuracy: 0.5097
Epoch 4/10
379/379 [==============================] - 22s 59ms/step - loss: 0.6932 - accuracy: 0.5058 - val_loss: 0.6939 - val_accuracy: 0.5002
Epoch 5/10
379/379 [==============================] - 21s 56ms/step - loss: 0.6925 - accuracy: 0.5118 - val_loss: 0.6932 - val_accuracy: 0.5108
Epoch 6/10
379/379 [==============================] - 22s 58ms/step - loss: 0.6922 - accuracy: 0.5103 - val_loss: 0.6919 - val_accuracy: 0.5126
Epoch 7/10
379/379 [==============================] - 23s 60ms/step - loss: 0.6907 - accuracy: 0.5141 - val_loss: 0.6921 - val_accuracy:

In [29]:
X_train.shape

(30274, 10, 49)